<!-- NOTEBOOK_METADATA source: "⚠️ Jupyter Notebook" title: "Monitor Exa AI Search with Langfuse" sidebarTitle: "Exa" logo: "/images/integrations/exa_icon.png" description: "Learn how to trace Exa AI search operations using Langfuse to capture detailed observability data for your search queries and API calls." category: "Integrations" -->

# Exa Integration

In this guide, we'll show you how to integrate [Langfuse](https://langfuse.com) with [Exa](https://exa.ai/) to trace your AI search operations. By leveraging Langfuse's tracing capabilities, you can automatically capture details such as inputs, outputs, and execution times of your Exa search functions.

> **What is Exa?** [Exa](https://exa.ai/) is an AI-powered search API built for LLMs and AI applications. Unlike traditional search engines, Exa is designed to understand semantic meaning and retrieve high-quality, relevant results that are perfect for AI use cases like RAG (Retrieval-Augmented Generation), research, and content discovery.

> **What is Langfuse?** [Langfuse](https://langfuse.com) is an open source LLM engineering platform that helps teams trace API calls, monitor performance, and debug issues in their AI applications.

## Get Started

First, install the necessary Python packages:

In [ ]:
%pip install langfuse exa-py

Next, configure your environment with your Langfuse API keys. You can obtain your keys from your Langfuse dashboard.

In [ ]:
import os
# Get keys for your project from the project settings page https://cloud.langfuse.com

os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..."
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..." 
os.environ["LANGFUSE_BASE_URL"] = "https://cloud.langfuse.com" # 🇪🇺 EU region
# os.environ["LANGFUSE_BASE_URL"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region

Initialize the Exa client with your Exa API key.

In [ ]:
from exa_py import Exa

exa = Exa(api_key="exa_...")

## Trace Exa Search Operations

To monitor your Exa search operations, we use the [Langfuse `@observe()` decorator](https://langfuse.com/docs/sdk/python/decorators). In this example, the `@observe()` decorator captures the inputs, outputs, and execution time of the `search_with_exa()` function. All trace data is automatically sent to Langfuse, allowing you to monitor your Exa operations in real time.

In [ ]:
from langfuse import observe

@observe()
def search_with_exa(query: str, num_results: int = 5):
    """Search the web using Exa AI and return results."""
    results = exa.search_and_contents(
        query,
        num_results=num_results,
        text=True
    )
    return results

# Example: Search for information about Langfuse
search_results = search_with_exa("What is Langfuse and how does it help with LLM observability?")

# Display the results
for result in search_results.results:
    print(f"Title: {result.title}")
    print(f"URL: {result.url}")
    print(f"Text: {result.text[:200]}...\n")

## Advanced: Trace Multiple Exa Operations

You can also trace more complex workflows that involve multiple Exa API calls. Here's an example that demonstrates searching and finding similar content:

In [ ]:
@observe()
def find_similar_content(url: str, num_results: int = 5):
    """Find similar content to a given URL using Exa."""
    results = exa.find_similar_and_contents(
        url,
        num_results=num_results
    )
    return results

@observe()
def search_and_find_similar(query: str):
    """Search for content and find similar pages."""
    # First, search for the query
    search_results = search_with_exa(query, num_results=3)
    
    # Then, find similar content to the top result
    if search_results.results:
        top_url = search_results.results[0].url
        similar_results = find_similar_content(top_url, num_results=3)
        return {
            "search_results": search_results,
            "similar_results": similar_results
        }
    
    return {"search_results": search_results}

# Execute the nested operations
results = search_and_find_similar("LLM observability tools")

## See Traces in Langfuse

After executing the traced functions, log in to your [Langfuse Dashboard](https://cloud.langfuse.com) to view detailed trace logs. You'll be able to see:

- Search queries and their parameters
- Response times for each API call
- Nested traces showing the relationship between search and similarity operations
- Full input and output data for debugging

The trace will show the complete execution flow, including all nested function calls and their respective inputs and outputs.

<!-- MARKDOWN_COMPONENT name: "LearnMore" path: "@/components-mdx/integration-learn-more.mdx" -->